In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import warnings
import sys
import time
warnings.filterwarnings('ignore')

from tqdm import tqdm
from pathlib import Path
from collections import Counter, defaultdict
from PIL import Image

In [2]:
try:
    import google.colab
    from google.colab import drive
    !uv pip install anomalib
    !uv pip install open-clip-torch
    drive.mount('/content/drive', force_remount=True)
    PROJECT_ROOT = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation') # 본인 경로 수정: Mac/Window
except ImportError:
    PROJECT_ROOT = Path.cwd().parents[1]

os.chdir(PROJECT_ROOT) # 현재 경로 수정
print(f"Current working directory: {os.getcwd()}")

Using Python 3.12.12 environment at: /usr
Resolved 94 packages in 1.47s
Prepared 12 packages in 534ms
Installed 12 packages in 16ms
 + anomalib==2.2.0
 + freia==0.2
 + imagecodecs==2026.1.14
 + jsonargparse==4.46.0
 + kornia==0.8.2
 + kornia-rs==0.1.10
 + lightning==2.6.1
 + lightning-utilities==0.15.2
 + pytorch-lightning==2.6.1
 + rich-argparse==1.7.2
 + torchmetrics==1.8.2
 + typeshed-client==2.8.2
Using Python 3.12.12 environment at: /usr
Resolved 49 packages in 212ms
Prepared 2 packages in 107ms
Installed 2 packages in 4ms
 + ftfy==6.3.1
 + open-clip-torch==3.2.0
Mounted at /content/drive
Current working directory: /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation


In [3]:
# TODO: PATH 설정

# dataset path
DATA_ROOT = PROJECT_ROOT / "dataset" / "MMAD"
DOMAIN_JSON = DATA_ROOT / "domain_knowledge.json"
MMAD_JSON = DATA_ROOT / "mmad.json"
META_CSV = DATA_ROOT / "metadata.csv"

# config.yaml path
CONFIG_ROOT = PROJECT_ROOT / "configs"
RUNTIME_CONFIG_ROOT = CONFIG_ROOT / "runtime.yaml"
EVAL_CONFIG_ROOT = CONFIG_ROOT / "eval.yaml"

# output path
OUTPUT_ROOT = PROJECT_ROOT / "output"

# check (선택사항)
# print(f"Project Root: {PROJECT_ROOT}")
# print(f"Data Root: {DATA_ROOT}")
# print(f"Config Root: {CONFIG_ROOT}")

### MVTecAD Dataset

In [13]:
import os
import pandas as pd
from pathlib import Path
from anomalib.data import Folder
from IPython.display import display

# 1. 경로 설정
PROJECT_ROOT = Path('/content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation')
base_data_path = PROJECT_ROOT / "dataset" / "MMAD"
DATASET_NAMES = ["GoodsAD", "MVTec-AD", "MVTec-LOCO", "VisA"]

def verify_all_datasets():
    if not base_data_path.exists():
        print(f"❌ 에러: {base_data_path} 경로가 존재하지 않습니다.")
        return

    all_results = []

    print(f"🚀 [Anomalib 2.2.0] 데이터셋 검증 시작")
    print(f"📍 경로: {base_data_path.resolve()}")
    print("-" * 70)

    for ds_name in DATASET_NAMES:
        ds_path = base_data_path / ds_name
        if not ds_path.exists():
            print(f"⚠️  [SKIP] {ds_name} 폴더가 없습니다.")
            continue

        categories = sorted([f.name for f in ds_path.iterdir() if f.is_dir()])

        for cat in categories:
            cat_path = ds_path / cat

            try:
                # -------------------------------------------------------
                # [Anomalib 2.2.0 핵심 수정 사항]
                # 1. validation_split_ratio -> split_ratio로 통합 관리되는 경우가 많음
                # 2. test_split_mode -> 인자 이름 확인 (v2.2는 test_split_mode 지원)
                # 3. extensions -> 리스트 형태로 전달
                # -------------------------------------------------------
                datamodule = Folder(
                    name=cat,
                    root=str(cat_path),
                    normal_dir="train/good",
                    test_split_mode="from_dir",
                    extensions=[".jpg", ".jpeg", ".png", ".JPG", ".PNG"],
                    train_batch_size=1,
                )

                datamodule.setup()

                # 데이터 장수 계산 (train_data 내에 val이 포함되어 있을 수 있음)
                train_num = len(datamodule.train_data) if datamodule.train_data is not None else 0
                val_num = len(datamodule.val_data) if datamodule.val_data is not None else 0
                test_num = len(datamodule.test_data) if datamodule.test_data is not None else 0

                all_results.append({
                    "Dataset": ds_name,
                    "Class": cat,
                    "Train(학습)": train_num,
                    "Val(검증)": val_num,
                    "Total_Good": train_num + val_num,
                    "Test(평가)": test_num,
                    "Status": "✅ OK"
                })

            except Exception as e:
                # 에러 발생 시 어떤 인자가 문제인지 정확히 보기 위해 에러 메시지 보존
                all_results.append({
                    "Dataset": ds_name,
                    "Class": cat,
                    "Train(학습)": 0, "Val(검증)": 0, "Total_Good": 0, "Test(평가)": 0,
                    "Status": f"❌ Error: {str(e)}"
                })

    if all_results:
        df = pd.DataFrame(all_results)
        print("\n📊 데이터셋 로드 결과 요약")
        display(df)

        print(f"\n✅ 검증 완료: 총 {len(df)}개 클래스 탐색")
        print(f"📈 정상 로드된 총 학습 가능 이미지: {df['Total_Good'].sum()} 장")

        errors = df[df['Status'].str.contains("❌")]
        if not errors.empty:
            print("\n🚨 에러 발생 클래스 상세:")
            display(errors)
    else:
        print("❌ 검색된 데이터가 없습니다.")

# 실행
verify_all_datasets()

🚀 [Anomalib 2.2.0] 데이터셋 검증 시작
📍 경로: /content/drive/.shortcut-targets-by-id/1MOlF1Xwaw_0p4R5EY_HXDRgLg0it43Jy/MMAD
----------------------------------------------------------------------



📊 데이터셋 로드 결과 요약


,Dataset,Class,Train(학습),Val(검증),Total_Good,Test(평가),Status
0,GoodsAD,cigarette_box,147,18,165,18,✅ OK
1,GoodsAD,drink_bottle,587,73,660,73,✅ OK
2,GoodsAD,drink_can,188,23,211,24,✅ OK
3,GoodsAD,food_bottle,812,101,913,101,✅ OK
4,GoodsAD,food_box,346,43,389,43,✅ OK
5,GoodsAD,food_package,432,54,486,54,✅ OK
6,MVTec-AD,bottle,168,20,188,21,✅ OK
7,MVTec-AD,cable,180,22,202,22,✅ OK
8,MVTec-AD,capsule,176,21,197,22,✅ OK
9,MVTec-AD,carpet,224,28,252,28,✅ OK



✅ 검증 완료: 총 38개 클래스 탐색
📈 정상 로드된 총 학습 가능 이미지: 15484 장


In [4]:
import anomalib.data
print(dir(anomalib.data))

['ADAM3D', 'ADAM3DDataset', 'AnomalibDataModule', 'AnomalibDataset', 'Avenue', 'AvenueDataset', 'BMAD', 'BMADDataset', 'BTech', 'BTechDataset', 'Batch', 'DataFormat', 'DatasetItem', 'Datumaro', 'DatumaroDataset', 'DepthBatch', 'DepthDataFormat', 'DepthItem', 'DictConfig', 'Enum', 'Folder', 'Folder3D', 'Folder3DDataset', 'FolderDataset', 'ImageBatch', 'ImageDataFormat', 'ImageItem', 'InferenceBatch', 'Kolektor', 'KolektorDataset', 'ListConfig', 'MPDD', 'MPDDDataset', 'MVTec', 'MVTec3D', 'MVTec3DDataset', 'MVTecAD', 'MVTecAD2', 'MVTecADDataset', 'MVTecLOCO', 'MVTecLOCODataset', 'NumpyImageBatch', 'NumpyImageItem', 'NumpyVideoBatch', 'NumpyVideoItem', 'PredictDataset', 'RealIAD', 'ShanghaiTech', 'ShanghaiTechDataset', 'Tabular', 'TabularDataset', 'UCSDped', 'UCSDpedDataset', 'UnknownDatamoduleError', 'VAD', 'VADDataset', 'VideoBatch', 'VideoDataFormat', 'VideoItem', 'Visa', 'VisaDataset', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package_

In [ ]:
from src.utils import load_config, load_json, load_csv
from anomalib.data import MVTecAD
from anomalib.models import EfficientAd, MVTecLOCO, Visa
from anomalib.engine import Engine

# TODO: runtime_config.yaml 확장/수정
runtime_config = load_config(RUNTIME_CONFIG_ROOT)
domain_json = load_json(DOMAIN_JSON)
mmad_json = load_json(MMAD_JSON)
meta_csv = load_csv(META_CSV)

# tqdm bar off
os.environ["TQDM_DISABLE"] = "1"

In [ ]:
!python /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation/notebooks/noh/train_efficientad.py --project-root /content/drive/Othercomputers/my_notebook/lion_final_pro_multimodal-anomaly-report-generation --category all --max-epochs 30

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
                                                               train_ae_epoch:  
                                                               0.543            
                                                               train_stae_epoch:
                                                               0.074            
                                                               train_loss_epoch:
Epoch 26/29 ━━━━╺━━━━━━━━━━━━ 11/42 0:00:02 • 0:00:07 5.14it/s train_st_step:   
                                                               3.081            
                                                               train_ae_step:   
                                                               0.593            
                                                               train_stae_step: 
                                                               0.078            
                                                               train_loss

In [ ]:
import logging
logging.disable(logging.WARNING)

# # tqdm bar on
# os.environ.pop("TQDM_DISABLE", None)

categories = [
    "bottle", "cable", "capsule", "carpet", "grid",
    "hazelnut", "leather", "metal_nut", "pill", "screw",
    "tile", "toothbrush", "transistor", "wood", "zipper"
]

all_predictions = {}
for i, category in enumerate(categories, 1):
    print(f"[{i}/{len(categories)}] Inference: {category}")
    # ckpt_path = OUTPUT_ROOT / category / "v0/weights/lightning/model.ckpt"
    ckpt_path = OUTPUT_ROOT / "EfficientAd" / "MVTecAD" / category / "v0/weights/lightning/model.ckpt"

    datamodule = MVTecAD(
          root=DATA_ROOT / "MVTec-AD",
          category=category
    )

    model = EfficientAd()
    engine = Engine(
        logger=False,
        enable_progress_bar=False,
        accelerator="auto",
        devices=1,
        default_root_dir=OUTPUT_ROOT
    )

    predictions = engine.predict(
        datamodule=datamodule,
        model=model,
        ckpt_path=ckpt_path,
    )

    all_predictions[category] = predictions
    print(f"✓ [{i}/{len(categories)}] {category} 완료\n")

In [ ]:
from src.visual.plot import kde_plot

y_true_list = []
y_score_list = []

for p in predictions:
    gt = p.gt_label.cpu().numpy() if hasattr(p.gt_label, 'cpu') else p.gt_label
    score = p.pred_score.cpu().numpy() if hasattr(p.pred_score, 'cpu') else p.pred_score
    y_true_list.append(gt)
    y_score_list.append(score)

y_true = np.concatenate(y_true_list)
y_score = np.concatenate(y_score_list)

normal_scores = y_score[y_true == 0]
anomaly_scores = y_score[y_true == 1]

scores_df = pd.DataFrame({
    'score': np.concatenate([normal_scores, anomaly_scores]),
    'label': ['Normal'] * len(normal_scores) + ['Anomaly'] * len(anomaly_scores)
})

kde_plot(
    scores_df,
    col='score',
    hue='label',
    palette=['steelblue', 'salmon'],
    title=f'{category}: Score Distribution',
    xlabel='Anomaly Score'
)

In [ ]:
import torch
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    jaccard_score  # IoU
)

results = []
for category, preds in all_predictions.items():
    # Image-level
    y_true = np.concatenate([p.gt_label.cpu().numpy() for p in preds])
    y_score = np.concatenate([p.pred_score.cpu().numpy() for p in preds])
    y_pred = (y_score >= 0.5).astype(int)

    # Pixel-level
    gt_masks = torch.cat([p.gt_mask for p in preds]).int()
    pred_masks = torch.cat([(p.anomaly_map > 0.5).int() for p in preds])

    metrics = {
        "Category": category,
        "AUROC": round(roc_auc_score(y_true, y_score), 4),
        "Accuracy": round(accuracy_score(y_true, y_pred), 4),
        "Precision": round(precision_score(y_true, y_pred, zero_division=0), 4),
        "Recall": round(recall_score(y_true, y_pred, zero_division=0), 4),
        "F1": round(f1_score(y_true, y_pred, zero_division=0), 4),
        "Dice": round(
            f1_score(
                gt_masks.flatten().cpu().numpy(),
                pred_masks.flatten().cpu().numpy(),
                zero_division=0
            ),
            4
        ),
        "IoU": round(
            jaccard_score(
                gt_masks.flatten().numpy(),
                pred_masks.flatten().numpy(),
                average='binary',
                zero_division=0
            ),
            4
        ),
        "N_samples": len(y_true)
    }
    results.append(metrics)

metrics_df = pd.DataFrame(results).set_index("Category")
avg_row = metrics_df.drop(columns=['N_samples']).mean().round(4)
avg_row['N_samples'] = metrics_df['N_samples'].sum()
metrics_df.loc['Average'] = avg_row

metrics_df

In [ ]:
from src.visual.plot import heatmap_plot

metrics_trans = metrics_df.drop('Average').drop(columns='N_samples')
heatmap_plot(
    metrics_trans,
    figsize=(10, 10),
    cmap='RdYlGn',
    annot=True,
    fmt='.4f',
    linewidths=0.5,
    title='PatchCore Performance by Category',
    rotation_x=45,
    rotation_y=0
)

In [ ]:
from src.visual.plot import anomaly_grid_from_dir

for category in categories:
    OUTPUT_IMG_ROOT = OUTPUT_ROOT / "Patchcore" / "MVTecAD" / category / "latest" / "images"
    print(f"{category}")
    anomaly_grid_from_dir(OUTPUT_IMG_ROOT, n_samples=1, n_cols=1)